In [1]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.pipeline import make_pipeline
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

In [2]:
df = pd.read_csv('train_prepared.csv')
y = df['17']
x = df[['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16']]
x_train, x_test_val, y_train, y_test_val = train_test_split(x, y, test_size=0.2)
df.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
count,8693.000000,8693.000000,8693.000000,8693.000000,8693.000000,8693.000000,8693.000000,8693.000000,8693.000000,8693.000000,8693.000000,8693.000000,8693.000000,8693.000000,8693.000000,8693.000000,8693.000000,8693.000000
mean,0.541355,0.250201,0.208444,0.210974,0.094444,0.694582,0.569999,69.306684,28.810882,0.023352,169.712297,401.258484,131.730128,270.398367,262.231451,593.757851,81.514667,0.503624
std,0.498315,0.433154,0.406219,0.408024,0.292462,0.460611,0.495104,1.815285,14.339536,0.151028,602.708957,1510.495548,560.917185,1093.010279,1094.438309,510.864150,1.500015,0.500016
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,65.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,80.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,68.000000,20.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,163.000000,80.000000,0.000000
50%,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,70.000000,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,417.000000,83.000000,1.000000
75%,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,71.000000,37.000000,0.000000,5.000000,22.000000,4.000000,22.000000,15.000000,987.000000,83.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,84.000000,79.000000,1.000000,14327.000000,29813.000000,23492.000000,22408.000000,24133.000000,1894.000000,83.000000,1.000000


In [5]:
random_forest = RandomForestClassifier(criterion='gini' , n_estimators=800, random_state=42)
xgb = XGBClassifier(n_estimators=1000, max_depth=4, learning_rate=0.05, colsample_bytree=0.7, random_state=42)
ensemble_model = XGBClassifier(base_estimator=random_forest, n_estimators=1000, max_depth=4, learning_rate=0.05, colsample_bytree=0.7, random_state=42)
ensemble_model.fit(x_train, y_train)

[11:23:20] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "base_estimator" } are not used.



XGBClassifier(base_estimator=RandomForestClassifier(n_estimators=800,
                                                    random_state=42),
              base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=1000, n_jobs=None, num_parallel_tree=None,
              predictor=None, ...)

In [6]:
y_pred = ensemble_model.predict(x_test_val)

In [7]:
accuracy = accuracy_score(y_test_val, y_pred)
print("Độ chính xác:", accuracy)

Độ chính xác: 0.7648073605520413


In [61]:
df = pd.read_csv('test.csv')
df = df[['PassengerId', 'HomePlanet']]

In [63]:
x_sub = pd.read_csv('test_prepared.csv')
x_sub
y_sub = xgb.predict(x_sub)
for row in range(len(df)):
    df.iloc[row, 1] = y_sub[row]
    if df.iloc[row, 1] == 0:
        df.iloc[row, 1] = False
    else:
        df.iloc[row, 1] = True
df = df.rename(columns={'HomePlanet': 'Transported'})
df.to_csv('submit.csv', index=False)